In [40]:
from pyhive import presto
import operator

def get_presto_conn():
    return presto.connect(host='presto-alpha-backend.data.houzz.net', port=8086)

def presto_query(query, presto_conn, fetchall=True):
    cursor = presto_conn.cursor()
    cursor.execute(query)
    header = list(map(operator.itemgetter(0), cursor.description))
    return header, cursor.fetchall()

conn = get_presto_conn() # establish the connection

# Or if want to use df format

import pandas as pd
from scipy.stats import ttest_ind
#df = pd.read_sql_query(q, conn)

In [83]:
q = '''
select *
from bls.blueshift_events
where dt >= '2021-12-15'
    and lower(campaign_name) like '%freq%'
    and lower(event_name) like '%deliver%'
'''

In [84]:
%%time
result = pd.read_sql_query(q, conn)

result

CPU times: user 1.26 s, sys: 294 ms, total: 1.56 s
Wall time: 32.6 s


,event_name,campaign_name,execution_key,user_id,email,url,event_id,bucket_uuid,campaign_uuid,experiment_uuid,message_uuid,trigger_uuid,creative_uuid,trigger_type,platform,http_header_user_agent,join_date,recent_active_date,idfa,idfv,ts,utm_medium,utm_source,utm_campaign,bsft_link_url,creative_name,device_type,os_type,campaign_type,communication_template,trigger_name,utm_content,email_subject,browser_type,dt
0,Email Delivered,Freq Bought Together - Return Purchaser - Email,None,366106,tammymorris4@gmail.com,None,016ef292-6fd6-4f7c-a3ea-d5552e3a7fd8,e418c4f4-fe7d-4698-93db-bba274361d06,fa7f3fb1-4e79-4464-a114-9d5503672f68,670c5157-cf5e-7b26-ee0c-c08c9fb91d7d,80f9e8c5-14b0-4ad3-99f8-9fee4beb7230,a71246c4-5055-484a-b064-439a2e8527db,fba66d88-17c1-4015-9e1e-c03bea373354,Email Trigger,None,None,2011-08-05 13:43:34.0,2022-01-02 15:49:36,None,None,1641165242000,email,blueshift,freq-bought-together-return-purchaser-email,None,Frequently Bought Together - Return Purchaser v2,Other,Other,transactional,Frequently Bought Together - Return Purchaser v2,Email,frequently-bought-together-return-purchaser-v2,You’ve got a good eye | We think you’ll also l...,None,2022-01-02
1,Email Delivered,Freq Bought Together - First Time Purchaser - ...,None,7457184,danabees3@gmail.com,None,9266f79b-da6e-4a6f-ad45-42777eb13e72,8e057752-79e2-4959-8e86-3e7677c30345,f2c3c26f-db71-4385-9d37-2c88457a82b5,88818570-a049-4d87-ba5a-2ed0b5aa1253,98fc565e-96ad-40d0-b0ce-81bad66f2cb8,f99bd916-2ff1-4c36-8547-fdccbee53b38,c3ad42c0-1af0-4d56-8b8a-b24f2d8d2ec0,Email Trigger,None,None,2013-05-26 18:24:26.0,None,None,None,1641168054000,email,blueshift,freq-bought-together-first-time-purchaser-email,None,Frequently Bought Together - First Time Purcha...,Other,Other,transactional,Frequently Bought Together - First Time Purcha...,Email,freq-bought-together-first-time-purchaser-email,Surprise! Here's a $25 gift towards your next ...,None,2022-01-02
2,Email Delivered,Freq Bought Together - First Time Purchaser - ...,None,12995142,gggnyc76@yahoo.com,None,bcef5459-a190-4d2d-af38-c10f014c93a5,8e057752-79e2-4959-8e86-3e7677c30345,f2c3c26f-db71-4385-9d37-2c88457a82b5,88818570-a049-4d87-ba5a-2ed0b5aa1253,3b29ae60-7d9c-44c6-961c-68d629010ee9,f99bd916-2ff1-4c36-8547-fdccbee53b38,c3ad42c0-1af0-4d56-8b8a-b24f2d8d2ec0,Email Trigger,None,None,2014-02-22 17:09:17.0,None,None,None,1641180567000,email,blueshift,freq-bought-together-first-time-purchaser-email,None,Frequently Bought Together - First Time Purcha...,Other,Other,transactional,Frequently Bought Together - First Time Purcha...,Email,freq-bought-together-first-time-purchaser-email,Surprise! Here's a $25 gift towards your next ...,None,2022-01-02
3,Email Delivered,Freq Bought Together - First Time Purchaser - ...,None,21974156,phoobearntigger@aol.com,None,7adbb9ed-9918-402b-888c-1b52ca39a3da,8e057752-79e2-4959-8e86-3e7677c30345,f2c3c26f-db71-4385-9d37-2c88457a82b5,88818570-a049-4d87-ba5a-2ed0b5aa1253,1a053720-57c9-4163-956d-827edb3748cf,f99bd916-2ff1-4c36-8547-fdccbee53b38,c3ad42c0-1af0-4d56-8b8a-b24f2d8d2ec0,Email Trigger,None,None,2015-02-12 15:04:23.0,None,None,None,1641114334000,email,blueshift,freq-bought-together-first-time-purchaser-email,None,Frequently Bought Together - First Time Purcha...,Other,Other,transactional,Frequently Bought Together - First Time Purcha...,Email,freq-bought-together-first-time-purchaser-email,Surprise! Here's a $25 gift towards your next ...,None,2022-01-02
4,Email Delivered,Freq Bought Together - Return Purchaser - Email,None,35048719,sfmbmg@hotmail.com,None,4adbfa0d-fd3a-46f8-b791-204039d306c1,e418c4f4-fe7d-4698-93db-bba274361d06,fa7f3fb1-4e79-4464-a114-9d5503672f68,f017b64f-3ae3-424e-9d80-b82f7852bdd4,5c20b24c-547f-4b10-9b88-c96ad88471ed,a71246c4-5055-484a-b064-439a2e8527db,51a6c982-ed23-4671-a7f8-74a97d6b18ef,Email Trigger,None,None,2016-08-31 13:23:07.0,None,None,None,1641157404000,email,blueshift,freq-bought-together-return-purchaser-email,None,Frequently Bought Together,Other,Other,transactional,

In [85]:
result[result['creative_name'].str.lower().str.contains('v2')].dt.min()

'2021-12-15'

In [86]:
result.groupby('campaign_name')['user_id'].nunique()

campaign_name
Freq Bought Together - First Time Purchaser - Email    11950
Freq Bought Together - First Time Purchaser - Push      1624
Freq Bought Together - Return Purchaser - Email         8436
Freq Bought Together - Return Purchaser - Push          2097
Name: user_id, dtype: int64

In [11]:
result.groupby('creative_name')['user_id'].nunique()

creative_name
Frequently Bought Together                                               5112
Frequently Bought Together - First Time Purchaser - Email                6090
Frequently Bought Together - First Time Purchaser - Email Reminder       4650
Frequently Bought Together - First Time Purchaser - Email V2             4226
Frequently Bought Together - First Time Purchaser - Email V2 Reminder    4564
Frequently Bought Together - First Time Purchaser - Push                  778
Frequently Bought Together - First Time Purchaser - Push V2               741
Frequently Bought Together - Return Purchaser - Push                     1228
Frequently Bought Together - Return Purchaser - Push V2                  1169
Frequently Bought Together - Return Purchaser v2                         4469
Name: user_id, dtype: int64

In [31]:
result[result['user_id']=='53912128']

,event_name,campaign_name,execution_key,user_id,email,url,event_id,bucket_uuid,campaign_uuid,experiment_uuid,message_uuid,trigger_uuid,creative_uuid,trigger_type,platform,http_header_user_agent,join_date,recent_active_date,idfa,idfv,ts,utm_medium,utm_source,utm_campaign,bsft_link_url,creative_name,device_type,os_type,campaign_type,communication_template,trigger_name,utm_content,email_subject,browser_type,dt
12932,Email Delivered,Freq Bought Together - Return Purchaser - Email,None,53912128,yuwei@houzz.com,None,a06c5c4e-58ca-42f5-a28b-9be7aca8d465,e418c4f4-fe7d-4698-93db-bba274361d06,fa7f3fb1-4e79-4464-a114-9d5503672f68,f017b64f-3ae3-424e-9d80-b82f7852bdd4,eaa9325a-1352-4a12-8e91-86760f71154d,a71246c4-5055-484a-b064-439a2e8527db,51a6c982-ed23-4671-a7f8-74a97d6b18ef,Email Trigger,None,None,2018-11-26 14:01:11.0,None,None,None,1640227431000,email,blueshift,freq-bought-together-return-purchaser-email,None,Frequently Bought Together,Other,Other,transactional,Frequently Bought Together,Email,freq-bought-together-return-purchaser-email,You’ve got a good eye | We think you’ll also l...,None,2021-12-22
12933,Email Delivered,Freq Bought Together - Return Purchaser - Email,None,53912128,yuwei@houzz.com,None,b3d09fa5-1af4-442c-84b8-929c6c57740b,e418c4f4-fe7d-4698-93db-bba274361d06,fa7f3fb1-4e79-4464-a114-9d5503672f68,670c5157-cf5e-7b26-ee0c-c08c9fb91d7d,9da6c6f3-4515-4c2e-a78c-3413f719b97e,a71246c4-5055-484a-b064-439a2e8527db,fba66d88-17c1-4015-9e1e-c03bea373354,Email Trigger,None,None,2018-11-26 14:01:11.0,None,None,None,1640227430000,email,blueshift,freq-bought-together-return-purchaser-email,None,Frequently Bought Together - Return Purchaser v2,Other,Other,transactional,Frequently Bought Together - Return Purchaser v2,Email,frequently-bought-together-return-purchaser-v2,You’ve got a good eye | We think you’ll also l...,None,2021-12-22
12934,Push Notification Delivered,Freq Bought Together - Return Purchaser - Push,None,53912128,yuwei@houzz.com,None,d108a3e6-e4f9-4986-8958-a7fa479f7b57,7fddb90b-d0ea-4de6-bba2-c8b462684b39,6dd00179-f120-4956-bcc6-1987f6189a66,53c90e57-5581-49bc-8588-c4cbc908b5a2,745edfe7-75e6-4912-8a7f-274a730b865a,4c3dd2a4-07be-4f17-b4be-e4057cf223a5,c20eed49-907b-480f-bbf7-f3ef79cdc8c4,Push Trigger,None,None,2018-11-26 14:01:11.0,None,None,847b59e7-e8fe-4ba1-bdb1-94f12fdf4497,1640227746000,push-message,blueshift,freq-bought-together-return-purchaser-push,None,Frequently Bought Together - Return Purchaser ...,Other,Other,transactional,Frequently Bought Together - Return Purchaser ...,Push,freq-bought-together-return-purchaser-push,None,None,2021-12-22
12935,Push Notification Delivered,Freq Bought Together - Return Purchaser - Push,None,53912128,yuwei@houzz.com,None,d63c52e4-81b2-4194-a044-cb6345fc1a25,7fddb90b-d0ea-4de6-bba2-c8b462684b39,6dd00179-f120-4956-bcc6-1987f6189a66,7533607f-7745-9a5f-feb6-59f99371f035,e12bc653-87dd-4a42-9661-909dfe8b3d8d,4c3dd2a4-07be-4f17-b4be-e4057cf223a5,65641fc2-3809-47e1-9585-041e45c16cbc,Push Trigger,None,None,2018-11-26 14:01:11.0,None,None,847b59e7-e8fe-4ba1-bdb1-94f12fdf4497,1640227747000,push-message,blueshift,freq-bought-together-return-purchaser-push,None,Frequently Bought Together - Return Purchaser ...,Other,Other,transactional,Frequently Bought Together - Return Purchaser ...,Push,frequently-bought-together-return-purchaser-pu...,None,None,2021-12-22
21001,Email Delivered,Freq Bought Together - Return Purchaser - Email,None,53912128,yuwei@houzz.com,None,2be7d579-025f-4401-a63e-5daa1317bfe3,e418c4f4-fe7d-4698-93db-bba274361d06,fa7f3fb1-4e79-4464-a114-9d5503672f68,670c5157-cf5e-7b26-ee0c-c08c9fb91d7d,6a733447-0646-42c5-ac2b-03a2f16ef1e8,a71246c4-5055-484a-b064-439a2e8527db,fba66d88-17c1-4015-9e1e-c03bea373354,Email Trigger,None,None,2018-11-26 14:01:11.0,None,None,None,1639639246000,email,blueshift,freq-bought-together-return-purchaser-email,None,Frequently Bought Together - Return Purchaser v2,Other,Other,transactional,Frequently Bought Together - Return Purchaser v2,Email,frequen

In [17]:
result[result['user_id']=='53912128']['dt'].value_counts()

2021-12-15    7
2021-12-18    7
2021-12-17    4
2021-12-22    4
2021-12-16    2
Name: dt, dtype: int64

In [16]:
result[result['user_id']=='53912128']['creative_name'].value_counts()

Frequently Bought Together                                 9
Frequently Bought Together - Return Purchaser - Push       5
Frequently Bought Together - Return Purchaser - Push V2    5
Frequently Bought Together - Return Purchaser v2           5
Name: creative_name, dtype: int64

In [30]:
result[result['creative_name']=='Frequently Bought Together']['campaign_name']

4        Freq Bought Together - Return Purchaser - Email
7        Freq Bought Together - Return Purchaser - Email
11       Freq Bought Together - Return Purchaser - Email
27       Freq Bought Together - Return Purchaser - Email
30       Freq Bought Together - Return Purchaser - Email
                              ...                       
35971    Freq Bought Together - Return Purchaser - Email
35975    Freq Bought Together - Return Purchaser - Email
35989    Freq Bought Together - Return Purchaser - Email
35990    Freq Bought Together - Return Purchaser - Email
35991    Freq Bought Together - Return Purchaser - Email
Name: campaign_name, Length: 6423, dtype: object

### Exclude users in both tests

In [32]:
q = '''
select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
'''

In [33]:
%%time
result = pd.read_sql_query(q, conn)

result

CPU times: user 31.1 ms, sys: 69.7 ms, total: 101 ms
Wall time: 9.76 s


,user_id
0,1874420
1,7190570
2,71187279
3,9099927
4,71100385
...,...
2248,62692593
2249,65369762
2250,57044608
2251,33498683


In [112]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-17'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-17'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,count(*) campaign_cnt
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.checkout_id) checkout_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.order_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1
'''

In [113]:
%%time
data = pd.read_sql_query(q, conn)

data

CPU times: user 59.3 ms, sys: 141 ms, total: 200 ms
Wall time: 35.4 s


,user_group,campaign_cnt,user_cnt,checkout_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,control,15210,11195,251,233,285,121144.98,177
1,treatment,14013,11071,280,258,334,136397.35,143


In [44]:
data.to_clipboard()

In [64]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-10'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-10'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,b.user_id
    ,count(distinct m.checkout_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1,2
'''

In [65]:
%%time
sample = pd.read_sql_query(q, conn)

sample

CPU times: user 114 ms, sys: 18.5 ms, total: 133 ms
Wall time: 40.5 s


,user_group,user_id,order_cnt,gmv,unsub
0,control,53577715,0,NaN,0
1,control,41695105,0,NaN,0
2,control,66479567,0,NaN,0
3,control,34160662,0,NaN,0
4,control,68024184,0,NaN,0
...,...,...,...,...,...
19354,treatment,41563416,0,NaN,0
19355,treatment,65804003,0,NaN,0
19356,treatment,9728267,0,NaN,0
19357,treatment,71176571,0,NaN,0


In [66]:
sample['gmv'].fillna(0,inplace=True)

In [70]:
sample.groupby('user_group')[['gmv']].describe()

gmv                                                     
             count       mean         std  min  25%  50%  75%       max
user_group                                                             
control     9771.0  10.040368  126.926913  0.0  0.0  0.0  0.0   6336.00
treatment   9588.0  11.705713  211.268087  0.0  0.0  0.0  0.0  14819.46

In [12]:
from scipy.stats import ttest_ind

In [68]:
ttest_ind(sample[sample['user_group']=='control']['gmv'],sample[sample['user_group']=='treatment']['gmv']).pvalue

0.5052728270122283

In [69]:
ttest_ind(sample[sample['user_group']=='control']['order_cnt'],sample[sample['user_group']=='treatment']['order_cnt']).pvalue

0.3953283314703234

In [61]:
sample_summary = sample.groupby(['user_group','user_id'])[['order_cnt','gmv']].sum().reset_index()

In [62]:
ttest_ind(sample_summary[sample_summary['user_group']=='control']['gmv'],
          sample_summary[sample_summary['user_group']=='treatment']['gmv']).pvalue

0.5052728270122283

In [63]:
ttest_ind(sample_summary[sample_summary['user_group']=='control']['order_cnt']
          ,sample_summary[sample_summary['user_group']=='treatment']['order_cnt']).pvalue

0.28923688570829836

In [79]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-10'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-10'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select dt
    ,user_group
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.checkout_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1,2
order by 1,2
'''

In [80]:
%%time
daily = pd.read_sql_query(q, conn)

daily

CPU times: user 60.2 ms, sys: 12.5 ms, total: 72.8 ms
Wall time: 36.6 s


,dt,user_group,user_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,2021-12-15,control,689,12,14,8622.92,7
1,2021-12-15,treatment,274,2,2,459.70,3
2,2021-12-16,control,122,2,2,1621.99,0
3,2021-12-16,treatment,116,2,2,3994.99,1
4,2021-12-17,control,1220,16,16,8383.31,25
5,2021-12-17,treatment,1246,24,25,4955.87,20
6,2021-12-18,control,396,11,12,5584.49,1
7,2021-12-18,treatment,395,9,10,4233.03,3
8,2021-12-19,control,449,6,6,1763.78,6
9,2021-12-19,treatment,483,8,9,4562.47,12


In [81]:
daily.to_clipboard()

In [29]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-19'
  and lower(b1.campaign_name) like '%freq%'
  and lower(b1.campaign_name) like '%first%'
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-19'
  and lower(b1.campaign_name) like '%freq%'
  and lower(b1.campaign_name) like '%first%'
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,count(*) campaign_cnt
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.checkout_id) checkout_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.order_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1
'''

In [30]:
%%time
data_first = pd.read_sql_query(q, conn)

data_first

CPU times: user 59.2 ms, sys: 13.4 ms, total: 72.6 ms
Wall time: 35.8 s


,user_group,campaign_cnt,user_cnt,checkout_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,control,10109,7694,127,123,136,50712.40,132
1,treatment,9549,7659,131,123,146,43504.84,104


In [31]:
data_first.to_clipboard()

In [32]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-19'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%first%'
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-19'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%first%'
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,b.user_id
    ,count(distinct m.checkout_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1,2
'''

In [33]:
%%time
sample_first = pd.read_sql_query(q, conn)

sample_first

CPU times: user 149 ms, sys: 27 ms, total: 176 ms
Wall time: 1min 11s


,user_group,user_id,order_cnt,gmv,unsub
0,treatment,71155171,0,NaN,0
1,treatment,18070976,0,NaN,0
2,treatment,35015714,0,NaN,0
3,treatment,17894697,0,NaN,0
4,treatment,22064454,0,NaN,0
...,...,...,...,...,...
15348,control,71091008,0,NaN,0
15349,control,71206919,0,NaN,0
15350,control,50192939,0,NaN,0
15351,control,71191412,0,NaN,0


In [34]:
sample_first['gmv'].fillna(0,inplace=True)

In [35]:
ttest_ind(sample_first[sample_first['user_group']=='control']['gmv']
          ,sample_first[sample_first['user_group']=='treatment']['gmv']).pvalue

0.4790836474357957

In [36]:
ttest_ind(sample_first[sample_first['user_group']=='control']['order_cnt']
          ,sample_first[sample_first['user_group']=='treatment']['order_cnt']).pvalue

0.7838248154691909

In [37]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-19'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%first%'
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-19'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%first%'
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select dt
    ,user_group
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.checkout_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1,2
order by 1,2
'''

In [38]:
%%time
daily_first = pd.read_sql_query(q, conn)

daily_first

CPU times: user 253 ms, sys: 48.4 ms, total: 302 ms
Wall time: 3min 32s


,dt,user_group,user_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,2021-12-15,control,409,1,1,65.98,5
1,2021-12-15,treatment,247,2,2,459.70,3
2,2021-12-16,control,89,2,2,1621.99,0
3,2021-12-16,treatment,85,1,1,34.99,1
4,2021-12-17,control,914,10,10,4439.96,22
...,...,...,...,...,...,...,...
67,2022-01-17,treatment,31,1,1,155.00,0
68,2022-01-18,control,41,3,3,1514.81,0
69,2022-01-18,treatment,31,1,1,882.00,0
70,2022-01-19,control,26,0,0,NaN,0


In [39]:
daily_first.to_clipboard()

In [41]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-31'
  and lower(b1.campaign_name) like '%freq%'
  and lower(b1.campaign_name) like '%return%'
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-31'
  and lower(b1.campaign_name) like '%freq%'
  and lower(b1.campaign_name) like '%return%'
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,count(*) campaign_cnt
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.checkout_id) checkout_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.order_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1
order by 1
'''

In [42]:
%%time
data_return = pd.read_sql_query(q, conn)

data_return

CPU times: user 66.2 ms, sys: 117 ms, total: 183 ms
Wall time: 39.1 s


,user_group,campaign_cnt,user_cnt,checkout_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,control,7878,5752,165,149,197,95921.16,97
1,treatment,7198,5570,232,204,290,144824.98,72


In [43]:
data_return.to_clipboard()

In [44]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-31'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%return%'
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-31'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%return%'
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,b.user_id
    ,count(distinct m.checkout_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1,2

'''

In [45]:
%%time
sample_return = pd.read_sql_query(q, conn)

sample_return

CPU times: user 88.1 ms, sys: 88 ms, total: 176 ms
Wall time: 28.5 s


,user_group,user_id,order_cnt,gmv,unsub
0,control,26469931,0,NaN,0
1,control,65146291,0,NaN,0
2,control,71145542,0,NaN,0
3,control,25366753,0,NaN,0
4,control,71043631,0,NaN,0
...,...,...,...,...,...
11317,treatment,49905962,0,NaN,0
11318,treatment,3019749,0,NaN,0
11319,treatment,71783001,0,NaN,0
11320,treatment,62344266,0,NaN,0


In [46]:
sample_return['gmv'].fillna(0,inplace=True)

In [47]:
ttest_ind(sample_return[sample_return['user_group']=='control']['gmv']
          ,sample_return[sample_return['user_group']=='treatment']['gmv']).pvalue

0.07801875191189567

In [48]:
ttest_ind(sample_return[sample_return['user_group']=='control']['order_cnt']
          ,sample_return[sample_return['user_group']=='treatment']['order_cnt']).pvalue

0.0009708041800630651

In [49]:
q = '''
with double_tests_users as (
-- exclude users in both v1 and v2
    select distinct b1.user_id
    from bls.blueshift_events b1
        inner join bls.blueshift_events b2
            on b1.user_id = b2.user_id
    where b1.dt >= '2021-12-15'
        and lower(b1.campaign_name) like '%freq%'
        and lower(b1.event_name) like '%deliver%'
        and lower(b1.creative_name) not like '%reminder%'
        and lower(b1.creative_name) not like '%v2%'
        and b2.dt >= '2021-12-15'
        and lower(b2.campaign_name) like '%freq%'
        and lower(b2.event_name) like '%deliver%'
        and lower(b2.creative_name) not like '%reminder%'
        and lower(b2.creative_name) like '%v2%'
),raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-31'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%return%'
  and lower(b1.creative_name) not like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment' as user_group
    ,b1.user_id
    ,b1.event_id
    ,b1.dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-12-15' and '2022-01-31'
  and lower(b1.campaign_name) like '%freq%' 
  and lower(b1.campaign_name) like '%return%'
  and lower(b1.creative_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
), preprocess as(
  select r1.*, r2.dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.event_id != r2.event_id
        and date_diff('day',date(r1.dt),date(r2.dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(dt) + interval '7' day as varchar)
      else cast(next_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select dt
    ,user_group
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.checkout_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-12-15'
where b.user_id not in (
              select user_id
              from double_tests_users
            )
group by 1,2
order by 1,2
'''

In [50]:
%%time
daily_return = pd.read_sql_query(q, conn)

daily_return

CPU times: user 52.6 ms, sys: 13.4 ms, total: 66 ms
Wall time: 32 s


,dt,user_group,user_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,2021-12-15,control,266,10,11,8210.58,2
1,2021-12-15,treatment,26,0,0,NaN,0
2,2021-12-16,control,29,0,0,NaN,0
3,2021-12-16,treatment,28,1,1,3960.00,0
4,2021-12-17,control,292,5,5,2411.49,3
...,...,...,...,...,...,...,...
91,2022-01-29,treatment,193,4,4,1767.09,3
92,2022-01-30,control,197,7,7,2227.04,5
93,2022-01-30,treatment,185,5,6,1347.74,2
94,2022-01-31,control,92,0,0,NaN,1


In [51]:
daily_return.fillna(0).to_clipboard()